In [6]:
import numpy as np
import matplotlib.pyplot as plt
from lora_modem import LoraModulator, LoraDemodulator, LoraSynchronizer


In [28]:
def get_sdr_info(group_number):
    if group_number <= 0 or group_number > 13:
        raise ValueError("Invalid group number")
    IPs = [-1, 'ip:192.168.1.31', 'ip:192.168.1.31', 'ip:192.168.1.32', 'ip:192.168.1.32', 'ip:192.168.1.33', 'ip:192.168.1.33', 'ip:192.168.1.34', 'ip:192.168.1.34', 'ip:192.168.1.35', 'ip:192.168.1.35', 'ip:192.168.1.36', 'ip:192.168.1.36', 'ip:192.168.1.37', 'ip:192.168.1.37']
    Carrier_freq = [-1, 910e6, 914e6, 918e6, 922e6, 926e6, 930e6, 934e6, 938e6, 942e6, 946e6, 950e6, 954e6, 958e6, 962e6]
    print(f"SDR for group {group_number}: \nIP: \"{IPs[group_number]}\" \nCarrier Frequency: {Carrier_freq[group_number]}")
    return IPs[group_number], Carrier_freq[group_number]

ip, CarrierFreq = get_sdr_info(9)

SDR for group 9: 
IP: "ip:192.168.1.35" 
Carrier Frequency: 942000000.0


In [3]:
import adi

#------------------------------- SDR Parameter Configuration -------------------------------

Uri              = ip            # RELLENAR CON LA IP DEL SDR ASIGNADO
SamplingRate     = 2e6           # Sample rate RX and TX paths[Samples/Sec]
Loopback         = 1             # 0=Disabled, 1=Digital, 2=RF

TxLOFreq         = CarrierFreq         # Carrier frequency of TX path [Hz] RELLENAR CON LA FREC. DE PORTADORA ASIGNADA
TxAtten          = -10           # Attenuation applied to TX path, valid range is -90 to 0 dB [dB]
TxRfBw           = 4e6           # Bandwidth of front-end analog filter of TX path [Hz]

RxLOFreq         = TxLOFreq      # Carrier frequency of RX path [Hz]
GainControlModes = "slow_attack" # Receive path AGC Options: slow_attack, fast_attack, manual
RxHardwareGain   = 0             # Gain applied to RX path. Only applicable when gain_control_mode is set to 'manual'    
RxRfBw           = TxRfBw        # Bandwidth of front-end analog filter of RX path [Hz] 
RxBufferSize     = 2**20-1

#------------------------------- SDR Setup ------------------------------- 

sdr             = adi.Pluto(Uri)
sdr.sample_rate = SamplingRate
sdr.loopback    = Loopback
#Tx:
sdr.tx_lo                 = int(TxLOFreq)
sdr.tx_hardwaregain_chan0 = TxAtten 
sdr.tx_rf_bandwidth       = int(TxRfBw)
sdr.tx_cyclic_buffer      = True
# Rx:
sdr.rx_lo                   = int(RxLOFreq)
sdr.gain_control_mode_chan0 = GainControlModes
sdr.rx_rf_bandwidth         = int(RxRfBw)
sdr.rx_cyclic_buffer        = False
sdr.rx_buffer_size          = RxBufferSize

In [ ]:
# ------------------------------- Graphic Plot Generation -------------------------------

In [25]:
# ----- Playground -----
spreading_factor = 7
bandwidth = 125e3
samples_per_chip = 5
preamble_number = 8

modulator = LoraModulator(spreading_factor, bandwidth, samples_per_chip)
demodulator = LoraDemodulator(spreading_factor, bandwidth, samples_per_chip)
synchronizer = LoraSynchronizer(spreading_factor, samples_per_chip, demodulator, preamble_number)
payload = [10, 50, 93]


pkg_signal, payload_signal = modulator.modulate_n_plot_explicit_package(preamble_number, payload, use_preamble=False, return_payload_signal=True)


sdr.tx(pkg_signal * 2**14)

rx_buffer = sdr.rx() / 2**14

sync_rx_payload = synchronizer.synchronize_rx_buffer(rx_buffer)
synchronizer.plot_synchronization(rx_buffer, sync_rx_payload, payload_signal)

if sync_rx_payload is not None:
    received_symbols = demodulator.demodulate_n_plot_symbols(sync_rx_payload)
    print('Received symbols: ', received_symbols)


NameError: name 'sdr' is not defined

In [ ]:
# Since it is not possible to turn off Tx, it is configured to transmit at low power and on a different frequency than Rx.
sdr.tx_destroy_buffer()
sdr.tx_hardwaregain_chan0 = -89 
sdr.tx_lo                 = int(2400e6)
sdr.rx_lo                 = int(950e6)
sdr.tx(np.zeros(2048))
# Destroy radio object
del(sdr)